In [ ]:
import sys
sys.path.insert(0, '..')

import os.path as op

import pandas as pd
import numpy as np

import matplotlib
import matplotlib.pyplot as plt
matplotlib.rc('font', **{'family': 'serif', 'serif': ['Computer Modern']})
matplotlib.rc('text', usetex=True)
matplotlib.rcParams.update({'font.size': 16})

from load_paus_cat import load_paus_cat

from astropy.coordinates import SkyCoord

from paus_utils import w_central
from jpasLAEs.utils import flux_to_mag

from astropy.io import fits
from astropy.table import Table
from astropy.cosmology import Planck18 as cosmo
import astropy.units as u


In [ ]:
# selection = pd.read_csv('/home/alberto/almacen/PAUS_data/catalogs/LAE_selection.csv')
selection = fits.open('/home/alberto/almacen/PAUS_data/catalogs/PAUS_LAE_selection_visual_insp_AT.fits')[1].data

sel_coords = SkyCoord(selection['RA'], selection['DEC'], unit='deg')

In [ ]:
# vi_cat = fits.open('/home/alberto/almacen/PAUS_data/catalogs/LAE_selection_VI_hiZ_with_MUV.fits')
# vi_ref_ID = vi_cat[1].data['ref_id_1']
# vi_is_LAE = vi_cat[1].data['is_LAE_VI']
# vi_field = vi_cat[1].data['field_1']

# selection['is_LAE_VI'] = np.zeros_like(selection['ref_id']).astype(bool)

# for field_name in ['W1', 'W2', 'W3']:
#     this_vi_ref_ID = vi_ref_ID[vi_field == field_name]
#     this_vi_is_LAE = vi_is_LAE[vi_field == field_name]
#     LAE_IDs = this_vi_ref_ID[this_vi_is_LAE]
#     for thisid in LAE_IDs:
#         if thisid in np.array(selection['ref_id']):
#             selection['is_LAE_VI'][(selection['ref_id'] == thisid) & (selection['field'] == field_name)] = True

In [ ]:
u_flux = np.empty(len(selection))
g_flux = np.empty(len(selection))
r_flux = np.empty(len(selection))
i_flux = np.empty(len(selection))
z_flux = np.empty(len(selection))

for field_name in ['W1', 'W2', 'W3']:
    path_to_cat = [f'/home/alberto/almacen/PAUS_data/catalogs/PAUS_3arcsec_{field_name}_extinction_corrected.pq']
    cat = load_paus_cat(path_to_cat)

    this_cat_coords = SkyCoord(cat['RA'], cat['DEC'], unit='deg')

    this_ids, this_dist, _ = sel_coords.match_to_catalog_sky(this_cat_coords)

    mask_dist = this_dist.value < (3 / 3600)

    u_flux[mask_dist] = cat['flx'][-6][this_ids][mask_dist]
    g_flux[mask_dist] = cat['flx'][-5][this_ids][mask_dist]
    r_flux[mask_dist] = cat['flx'][-4][this_ids][mask_dist]
    i_flux[mask_dist] = cat['flx'][-3][this_ids][mask_dist]
    z_flux[mask_dist] = cat['flx'][-2][this_ids][mask_dist]

In [ ]:
# SDSS catalog
path_to_cat = '/home/alberto/almacen/SDSS_spectra_fits/DR18/spAll-v5_13_2.fits'
sdss_cat = fits.open(path_to_cat)[1].data
print('SDSS cat loaded.')

# HETDEX catalog
path_to_cat = '/home/alberto/almacen/HETDEX_catalogs/hetdex_source_catalog_1'
version = 'v3.2'
hetdex_cat = pd.DataFrame(
    dict(Table.read(op.join(path_to_cat, f'hetdex_sc1_{version}.ecsv')))
    )
print('HETDEX cat loaded.')

# DESI catalog
path_to_cat = '/home/alberto/almacen/DESI/zall-pix-fuji.fits'
desi_cat = fits.open(path_to_cat)[1].data
print('DESI cat loaded.')

In [ ]:
# Do the cross-matches
coords_paus = SkyCoord(ra=np.asarray(selection['RA']) * u.deg,
                       dec=np.asarray(selection['DEC']) * u.deg)
coords_sdss = SkyCoord(ra=sdss_cat['RA'] * u.deg,
                       dec=sdss_cat['DEC'] * u.deg)
coords_hetdex = SkyCoord(ra=np.asarray(hetdex_cat['RA']) * u.deg,
                         dec=np.asarray(hetdex_cat['DEC']) * u.deg)
coords_desi = SkyCoord(ra=desi_cat['TARGET_RA'] * u.deg,
                       dec=desi_cat['TARGET_DEC'] * u.deg)

xm_id_sdss, ang_dist_sdss, _= coords_paus.match_to_catalog_sky(coords_sdss)
xm_id_hetdex, ang_dist_hetdex, _= coords_paus.match_to_catalog_sky(coords_hetdex)
xm_id_desi, ang_dist_desi, _= coords_paus.match_to_catalog_sky(coords_desi)

# Objects with 1 arcsec of separation
mask_dist_sdss = (ang_dist_sdss <= 1.5 * u.arcsec) & (sdss_cat['ZWARNING'][xm_id_sdss] == 0)
mask_dist_desi = (ang_dist_desi <= 1.5 * u.arcsec) & (desi_cat['ZWARN'][xm_id_desi] == 0)
mask_dist_hetdex = (ang_dist_hetdex <= 1.5 * u.arcsec) & (hetdex_cat['z_hetdex_conf'][xm_id_hetdex] > 0.9)

mask_dist_sdss = np.asarray(mask_dist_sdss)
mask_dist_desi = np.asarray(mask_dist_desi)
mask_dist_hetdex = np.asarray(mask_dist_hetdex)

mask_spec = mask_dist_sdss | mask_dist_desi | mask_dist_hetdex

In [ ]:
u_mag = flux_to_mag(u_flux, w_central[-6])
g_mag = flux_to_mag(g_flux, w_central[-5])
r_mag = flux_to_mag(r_flux, w_central[-4])
i_mag = flux_to_mag(i_flux, w_central[-3])
z_mag = flux_to_mag(z_flux, w_central[-2])

In [ ]:
sdss_z_mask = (sdss_cat['ZWARNING'] == 0) & (sdss_cat['CLASS'] == 'QSO')
sdss_z_mask_star = (sdss_cat['ZWARNING'] == 0) & (sdss_cat['CLASS'] == 'STAR')

sdss_ugriz = 22.5 - 2.5 * np.log10(sdss_cat['SPECTROFLUX'])

sdss_z = sdss_cat['Z'][sdss_z_mask]

sdss_ug = sdss_ugriz[:, 0][sdss_z_mask] - sdss_ugriz[:, 1][sdss_z_mask]
sdss_gr = sdss_ugriz[:, 1][sdss_z_mask] - sdss_ugriz[:, 2][sdss_z_mask]
sdss_ri = sdss_ugriz[:, 2][sdss_z_mask] - sdss_ugriz[:, 3][sdss_z_mask]

sdss_ug_star = sdss_ugriz[:, 0][sdss_z_mask_star] - sdss_ugriz[:, 1][sdss_z_mask_star]
sdss_gr_star = sdss_ugriz[:, 1][sdss_z_mask_star] - sdss_ugriz[:, 2][sdss_z_mask_star]
sdss_ri_star = sdss_ugriz[:, 2][sdss_z_mask_star] - sdss_ugriz[:, 3][sdss_z_mask_star]

In [ ]:
desi_gr = -2.5*np.log10(desi_cat['FLUX_G']) + 2.5*np.log10(desi_cat['FLUX_R'])

In [ ]:
def contour_curve(xx, yy, bins_xx, bins_yy, levels=[0.683],
                  N_bins=10000):
    if np.any((np.asarray(levels) > 1) | (np.asarray(levels) <= 0)):
        raise Exception('Levels must be in the interval (0, 1]')

    Z, x, y = np.histogram2d(xx, yy, bins=(bins_xx, bins_yy))

    H_min = np.amin(Z)
    H_max = np.amax(Z)

    y_centers = 0.5 * (y[1:] + y[:-1])
    x_centers = 0.5 * (x[1:] + x[:-1])

    H_Arr = np.linspace(H_min, H_max, N_bins)[::-1]

    fact_up_Arr = np.zeros(N_bins)

    TOTAL_H = np.sum(Z)

    for iii in range(0, N_bins):
        mask = Z > H_Arr[iii]
        fact_up_Arr[iii] = np.sum(Z[mask]) / TOTAL_H

    out_levels = list()
    for lvl in levels:
        this_H_value = np.interp(lvl, fact_up_Arr, H_Arr)
        out_levels.append(this_H_value)

    return x_centers, y_centers, Z.T, out_levels

In [ ]:
z_min = 2.75
z_max = 3.5

r_max = 21

cbins = np.linspace(-2, 4, 50)

cmap_name = 'rainbow'

cmap = plt.get_cmap(cmap_name)

fig, axes = plt.subplots(2, 3, figsize=(6, 6),
                         width_ratios=[1, 0.3, 0.07],
                         height_ratios=[0.3, 1])

ax = axes[1, 0]
ax_hist_top = axes[0, 0]
ax_hist_right = axes[1, 1]
ax_cbar = axes[1, 2]
ax_empty_1 = axes[0, 1]
ax_empty_2 = axes[0, 2]


zmask_sdss = (sdss_z > z_min) & (sdss_z < z_max) & (sdss_ugriz[:, 2][sdss_z_mask] < r_max)
zmask_paus = (selection['z_NB'] > z_min) & (selection['z_NB'] < z_max) & selection['is_LAE_VI'] & (selection['r_mag'] < r_max)

MUV_min = -28.8
MUV_max = -25

paus_colors = (selection['M_UV'][zmask_paus] - MUV_min) / (MUV_max - MUV_min)
sc = ax.scatter((g_mag - r_mag)[zmask_paus], (r_mag - i_mag)[zmask_paus],
                c=paus_colors, s=20, cmap=cmap, vmin=0, vmax=1)

fig.colorbar(sc, cax=ax_cbar)

L_labels = np.arange(-30, -25, 1)
ax_cbar.set_yticks(np.interp(L_labels, np.linspace(MUV_min, MUV_max, 10), np.linspace(0, 1, 10)))
ax_cbar.set_yticklabels(L_labels)
ax_cbar.set_ylabel(r'$M_{\rm UV}$')
ax_cbar.invert_yaxis()

ax.scatter(sdss_gr[zmask_sdss], sdss_ri[zmask_sdss], rasterized=True, 
           zorder=-99, alpha=0.3, s=1, c='dimgray')

contour_args = contour_curve(sdss_gr[zmask_sdss], sdss_ri[zmask_sdss],
                             cbins, cbins, levels=[0.95, 0.68])
ax.contour(*contour_args, colors='k', zorder=-98)


ylims = (-1, 1.5)
xlims = (-1, 2)

ax.set_ylim(*ylims)
ax.set_xlim(*xlims)
ax_hist_top.set_xlim(*xlims)
ax_hist_right.set_ylim(*ylims)

ax.set_xlabel(r'$g-r$')
ax.set_ylabel(r'$r-i$')

ax_cbar.yaxis.tick_right()
ax_hist_top.set_xticklabels([])
ax_hist_right.set_yticklabels([])
ax_cbar.set_xticks([])

## Text redshift
ax_empty_1.text(0.15, 0.4, fr'${z_min:0.1f}<z<{z_max:0.1f}$',
                fontsize=17)

## Histograms

ax_hist_top.hist((g_mag - r_mag)[zmask_paus], bins=cbins,
                 lw=2, color='teal', density=True)
ax_hist_top.hist(sdss_gr[zmask_sdss], bins=cbins,
                 histtype='step', lw=2, color='k',
                 ls='-', density=True)

ax_hist_right.hist((r_mag - i_mag)[zmask_paus], bins=cbins,
                 lw=2, color='teal', density=True, orientation='horizontal')
ax_hist_right.hist(sdss_ri[zmask_sdss], bins=cbins,
                 histtype='step', lw=2, color='k',
                 ls='-', density=True, orientation='horizontal')

# Empty axes
for this_ax in [ax_empty_1, ax_empty_2]:
    this_ax.set_frame_on(False)
    this_ax.set_yticks([])
    this_ax.set_xticks([])

ax_hist_top.set_yticks([])
ax_hist_right.set_xticks([])

for this_ax in axes.flatten():
    this_ax.tick_params(direction='in', which='both')
    this_ax.yaxis.set_ticks_position('both')
    this_ax.xaxis.set_ticks_position('both')

fig.subplots_adjust(wspace=0.1, hspace=0.05)

plt.show()

In [ ]:
z_min = 2.7
z_max = 3.5

cbins = np.linspace(-2, 4, 50)

fig, axes = plt.subplots(2, 3, figsize=(6, 6),
                         width_ratios=[1, 0.3, 0.07],
                         height_ratios=[0.3, 1])

ax = axes[1, 0]
ax_hist_top = axes[0, 0]
ax_hist_right = axes[1, 1]
ax_cbar = axes[1, 2]
ax_empty_1 = axes[0, 1]
ax_empty_2 = axes[0, 2]


zmask_sdss = (sdss_z > z_min) & (sdss_z < z_max) & (sdss_ugriz[:, 2][sdss_z_mask] < 21)
zmask_paus = (selection['z_NB'] > z_min) & (selection['z_NB'] < z_max) & selection['is_LAE_VI'] & (selection['r_mag'] < 21)

paus_colors = (selection['M_UV'][zmask_paus] - MUV_min) / (MUV_max - MUV_min)
sc = ax.scatter((u_mag - g_mag)[zmask_paus], (g_mag - r_mag)[zmask_paus],
                c=paus_colors, s=20, cmap=cmap, vmin=0, vmax=1)

fig.colorbar(sc, cax=ax_cbar)

L_labels = np.arange(-30, -25, 1)
ax_cbar.set_yticks(np.interp(L_labels, np.linspace(MUV_min, MUV_max, 10), np.linspace(0, 1, 10)))
ax_cbar.set_yticklabels(L_labels)
ax_cbar.set_ylabel(r'$M_{\rm UV}$')
ax_cbar.invert_yaxis()

ax.scatter(sdss_ug[zmask_sdss], sdss_gr[zmask_sdss], rasterized=True, 
           zorder=-99, alpha=0.3, s=1, c='dimgray')

contour_args = contour_curve(sdss_ug[zmask_sdss], sdss_gr[zmask_sdss],
                             cbins, cbins, levels=[0.95, 0.68])
ax.contour(*contour_args, colors='k', zorder=-98)


ylims = (-0.5, 2)
xlims = (-1, 4)

ax.set_ylim(*ylims)
ax.set_xlim(*xlims)
ax_hist_top.set_xlim(*xlims)
ax_hist_right.set_ylim(*ylims)

ax.set_xlabel(r'$u-g$')
ax.set_ylabel(r'$g-r$')

ax_cbar.yaxis.tick_right()
ax_hist_top.set_xticklabels([])
ax_hist_right.set_yticklabels([])
ax_cbar.set_xticks([])

ax_hist_top.set_yticks([])
ax_hist_right.set_xticks([])


## Text redshift
ax_empty_1.text(0.15, 0.4, fr'${z_min:0.1f}<z<{z_max:0.1f}$',
                fontsize=17)

## Histograms

ax_hist_top.hist((u_mag - g_mag)[zmask_paus], bins=cbins,
                 lw=2, color='teal', density=True)
ax_hist_top.hist(sdss_ug[zmask_sdss], bins=cbins,
                 histtype='step', lw=2, color='k',
                 ls='-', density=True)

ax_hist_right.hist((g_mag - r_mag)[zmask_paus], bins=cbins,
                 lw=2, color='teal', density=True, orientation='horizontal')
ax_hist_right.hist(sdss_gr[zmask_sdss], bins=cbins,
                 histtype='step', lw=2, color='k',
                 ls='-', density=True, orientation='horizontal')

# Empty axes
for this_ax in [ax_empty_1, ax_empty_2]:
    this_ax.set_frame_on(False)
    this_ax.set_yticks([])
    this_ax.set_xticks([])

for this_ax in axes.flatten():
    this_ax.tick_params(direction='in', which='both')
    this_ax.yaxis.set_ticks_position('both')
    this_ax.xaxis.set_ticks_position('both')

fig.subplots_adjust(wspace=0.1, hspace=0.05)

plt.show()

In [ ]:
z_min = 3.5
z_max = 4.5

r_max = 21

cbins = np.linspace(-2, 4, 50)

cmap_name = 'rainbow'

cmap = plt.get_cmap(cmap_name)

fig, axes = plt.subplots(2, 3, figsize=(6, 6),
                         width_ratios=[1, 0.3, 0.07],
                         height_ratios=[0.3, 1])

ax = axes[1, 0]
ax_hist_top = axes[0, 0]
ax_hist_right = axes[1, 1]
ax_cbar = axes[1, 2]
ax_empty_1 = axes[0, 1]
ax_empty_2 = axes[0, 2]


zmask_sdss = (sdss_z > z_min) & (sdss_z < z_max) & (sdss_ugriz[:, 2][sdss_z_mask] < r_max)
zmask_paus = (selection['z_NB'] > z_min) & (selection['z_NB'] < z_max) & selection['is_LAE_VI'] & (selection['r_mag'] < r_max)

paus_colors = (selection['M_UV'][zmask_paus] - MUV_min) / (MUV_max - MUV_min)
sc = ax.scatter((g_mag - r_mag)[zmask_paus], (r_mag - i_mag)[zmask_paus],
                c=paus_colors, s=20, cmap=cmap, vmin=0, vmax=1)

fig.colorbar(sc, cax=ax_cbar)

L_labels = np.arange(-30, -25, 1)
ax_cbar.set_yticks(np.interp(L_labels, np.linspace(MUV_min, MUV_max, 10), np.linspace(0, 1, 10)))
ax_cbar.set_yticklabels(L_labels)
ax_cbar.set_ylabel(r'$M_{\rm UV}$')
ax_cbar.invert_yaxis()

ax.scatter(sdss_gr[zmask_sdss], sdss_ri[zmask_sdss], rasterized=True, 
           zorder=-99, alpha=0.3, s=1, c='dimgray')

contour_args = contour_curve(sdss_gr[zmask_sdss], sdss_ri[zmask_sdss],
                             cbins, cbins, levels=[0.95, 0.68])
ax.contour(*contour_args, colors='k', zorder=-98)


ylims = (-1, 1.5)
xlims = (-0.5, 3)

ax.set_ylim(*ylims)
ax.set_xlim(*xlims)
ax_hist_top.set_xlim(*xlims)
ax_hist_right.set_ylim(*ylims)

ax.set_xlabel(r'$g-r$')
ax.set_ylabel(r'$r-i$')

ax_cbar.yaxis.tick_right()
ax_hist_top.set_xticklabels([])
ax_hist_right.set_yticklabels([])
ax_cbar.set_xticks([])

## Text redshift
ax_empty_1.text(0.15, 0.4, fr'${z_min:0.1f}<z<{z_max:0.1f}$',
                fontsize=17)

## Histograms

ax_hist_top.hist((g_mag - r_mag)[zmask_paus], bins=cbins,
                 lw=2, color='teal', density=True)
ax_hist_top.hist(sdss_gr[zmask_sdss], bins=cbins,
                 histtype='step', lw=2, color='k',
                 ls='-', density=True)

ax_hist_right.hist((r_mag - i_mag)[zmask_paus], bins=cbins,
                 lw=2, color='teal', density=True, orientation='horizontal')
ax_hist_right.hist(sdss_ri[zmask_sdss], bins=cbins,
                 histtype='step', lw=2, color='k',
                 ls='-', density=True, orientation='horizontal')

# Empty axes
for this_ax in [ax_empty_1, ax_empty_2]:
    this_ax.set_frame_on(False)
    this_ax.set_yticks([])
    this_ax.set_xticks([])

for this_ax in axes.flatten():
    this_ax.tick_params(direction='in', which='both')
    this_ax.yaxis.set_ticks_position('both')
    this_ax.xaxis.set_ticks_position('both')

ax_hist_top.set_yticks([])
ax_hist_right.set_xticks([])

fig.subplots_adjust(wspace=0.1, hspace=0.05)

plt.show()

In [ ]:
z_min = 3.5
z_max = 4.5

r_max = 22.5

cbins = np.linspace(-2, 7, 50)
cbins_2 = np.linspace(-2, 4, 50)

fig, axes = plt.subplots(2, 3, figsize=(6, 6),
                         width_ratios=[1, 0.3, 0.07],
                         height_ratios=[0.3, 1])

ax = axes[1, 0]
ax_hist_top = axes[0, 0]
ax_hist_right = axes[1, 1]
ax_cbar = axes[1, 2]
ax_empty_1 = axes[0, 1]
ax_empty_2 = axes[0, 2]


zmask_sdss = (sdss_z > z_min) & (sdss_z < z_max) & (sdss_ugriz[:, 2][sdss_z_mask] < r_max)
zmask_paus = (selection['z_NB'] > z_min) & (selection['z_NB'] < z_max) & selection['is_LAE_VI'] & (selection['r_mag'] < r_max)

paus_colors = (selection['M_UV'][zmask_paus] - MUV_min) / (MUV_max - MUV_min)
sc = ax.scatter((u_mag - g_mag)[zmask_paus], (g_mag - r_mag)[zmask_paus],
                c=paus_colors, s=20, cmap=cmap, vmin=0, vmax=1)

fig.colorbar(sc, cax=ax_cbar)

L_labels = np.arange(-30, -25, 1)
ax_cbar.set_yticks(np.interp(L_labels, np.linspace(MUV_min, MUV_max, 10), np.linspace(0, 1, 10)))
ax_cbar.set_yticklabels(L_labels)
ax_cbar.set_ylabel(r'$M_{\rm UV}$')
ax_cbar.invert_yaxis()

ax.scatter(sdss_ug[zmask_sdss], sdss_gr[zmask_sdss], rasterized=True, 
           zorder=-99, alpha=0.3, s=1, c='dimgray')

contour_args = contour_curve(sdss_ug[zmask_sdss], sdss_gr[zmask_sdss],
                             cbins, cbins_2, levels=[0.95, 0.68])
ax.contour(*contour_args, colors='k', zorder=-98)


ylims = (-0.7, 3)
xlims = (-1, 7)

ax.set_ylim(*ylims)
ax.set_xlim(*xlims)
ax_hist_top.set_xlim(*xlims)
ax_hist_right.set_ylim(*ylims)

ax.set_xlabel(r'$u-g$')
ax.set_ylabel(r'$g-r$')

ax_cbar.yaxis.tick_right()
ax_hist_top.set_xticklabels([])
ax_hist_right.set_yticklabels([])
ax_cbar.set_xticks([])

## Text redshift
ax_empty_1.text(0.15, 0.4, fr'${z_min:0.1f}<z<{z_max:0.1f}$',
                fontsize=17)

## Histograms

ax_hist_top.hist((u_mag - g_mag)[zmask_paus], bins=cbins,
                 lw=2, color='teal', density=True)
ax_hist_top.hist(sdss_ug[zmask_sdss], bins=cbins,
                 histtype='step', lw=2, color='k',
                 ls='-', density=True)

ax_hist_right.hist((g_mag - r_mag)[zmask_paus], bins=cbins,
                 lw=2, color='teal', density=True, orientation='horizontal')
ax_hist_right.hist(sdss_gr[zmask_sdss], bins=cbins,
                 histtype='step', lw=2, color='k',
                 ls='-', density=True, orientation='horizontal')

# Empty axes
for this_ax in [ax_empty_1, ax_empty_2]:
    this_ax.set_frame_on(False)
    this_ax.set_yticks([])
    this_ax.set_xticks([])

for this_ax in axes.flatten():
    this_ax.tick_params(direction='in', which='both')
    this_ax.yaxis.set_ticks_position('both')
    this_ax.xaxis.set_ticks_position('both')

ax_hist_top.set_yticks([])
ax_hist_right.set_xticks([])

fig.subplots_adjust(wspace=0.1, hspace=0.05)

plt.show()

In [ ]:
z_min = 2.75
z_max = 3.5

zmask_sdss = (sdss_z > z_min) & (sdss_z < z_max)
zmask_desi = (desi_cat['Z'] > z_min) & (desi_cat['Z'] < z_max) & (desi_cat['ZWARN'] == 0) & (desi_cat['SPECTYPE'] == 'QSO')
zmask_paus = (selection['z_NB'] > z_min) & (selection['z_NB'] < z_max) & selection['is_LAE_VI']

cbins = np.linspace(-1.5, 2.5, 40)

fig, ax = plt.subplots(figsize=(6, 5))

ax.hist(sdss_gr[zmask_sdss], cbins, histtype='step',
        density=True, lw=2)
ax.hist(desi_gr[zmask_desi], cbins, histtype='step', 
        density=True, lw=2)
ax.hist((g_mag - r_mag)[zmask_paus], cbins, histtype='step',
        density=True, lw=2)

plt.show()

In [ ]:
# And the line catalog
sdss_dir = '/home/alberto/almacen/SDSS_spectra_fits'
fread = fits.open(f'{sdss_dir}/spAllLine-v5_13_0.fits')
sdss_line_cat = fread[1].data[np.where(fread[1].data['LINEWAVE'] == 1215.67)]

In [ ]:
sdss_L_Lya = np.log10(sdss_line_cat['LINEAREA'] * 1e-17 * 4 * np.pi * cosmo.luminosity_distance(sdss_line_cat['LINEZ']).to(u.cm).value**2)

In [ ]:
ew_bins = np.linspace(10, 300, 20)

fig, ax = plt.subplots(figsize=(6, 5))

paus_mask = (selection['L_lya_corr'] > 43.5) & (selection['L_lya_corr'] < 44.5)
sdss_mask = (sdss_L_Lya > 43.5) & (sdss_L_Lya < 44.5) & (sdss_line_cat['LINEZ'] > 2.75)

ax.hist(selection['EW0_lya'][paus_mask], ew_bins,
        density=True, histtype='step', lw=2, color='crimson',
        label='PAUS')
ax.hist((sdss_line_cat['LINEEW'] / (1 + sdss_line_cat['LINEZ']))[sdss_mask], ew_bins,
        density=True, histtype='step', lw=2, color='teal',
        label='SDSS (DR16)')

# paus_mask = (selection['L_lya_corr'] > 44.5)
# sdss_mask = (sdss_L_Lya > 44.5) & (sdss_line_cat['LINEZ'] > 2.75)

# ax.hist(selection['EW0_lya'][paus_mask], ew_bins,
#         density=True, histtype='step', lw=2, color='crimson',
#         label='PAUS', ls='--')
# ax.hist((sdss_line_cat['LINEEW'] / (1 + sdss_line_cat['LINEZ']))[sdss_mask], ew_bins,
#         density=True, histtype='step', lw=2, color='teal',
#         label='SDSS (DR16)', ls='--')

ax.set_xlabel(r'Ly$\alpha$ EW$_0$ [\AA]')

ax.legend()

plt.show()

In [ ]:
pd16_g_LF_215 = pd.read_csv('/home/alberto/almacen/PAUS_data/literature_UV_LFs/PD16_g21.5_counts.csv')
pd16_g_LF_225 = pd.read_csv('/home/alberto/almacen/PAUS_data/literature_UV_LFs/PD16_g22.5_counts.csv')

z_bins = np.linspace(0, 6, 30)
delta_z = z_bins[1] - z_bins[0]

fig, ax = plt.subplots(figsize=(6, 5))

paus_mask = g_mag < 21.5
ax.hist(selection['z_NB'][paus_mask], z_bins,
        weights=np.ones(sum(paus_mask)) * 35**-1 / delta_z,
        histtype='step', lw=1.5, color='purple', ls='--')

paus_mask = (g_mag < 21.5) & selection['is_LAE_VI']
ax.hist(selection['z_NB'][paus_mask], z_bins,
        weights=np.ones(sum(paus_mask)) * 35**-1 / delta_z,
        histtype='step', lw=2, color='purple')
        
paus_mask = g_mag < 25
ax.hist(selection['z_NB'][paus_mask], z_bins,
        weights=np.ones(sum(paus_mask)) * 35**-1 / delta_z,
        histtype='step', lw=1.5, color='k', ls='--')

paus_mask = (g_mag < 25) & selection['is_LAE_VI']
ax.hist(selection['z_NB'][paus_mask], z_bins,
        weights=np.ones(sum(paus_mask)) * 35**-1 / delta_z,
        histtype='step', lw=2, color='k')
    
ax.plot([], [], c='k', lw=3,
        label='This work (Full sample)')
ax.plot([], [], c='peru', lw=3,
        label='This work ($g < 22.5$)')
ax.plot([], [], c='purple', lw=3,
        label='This work ($g < 21.5$)')

paus_mask = g_mag < 22.5
ax.hist(selection['z_NB'][paus_mask], z_bins,
        weights=np.ones(sum(paus_mask)) * 36**-1 / delta_z,
        histtype='step', lw=1.5, color='peru', ls='--')

paus_mask = (g_mag < 22.5) & selection['is_LAE_VI']
ax.hist(selection['z_NB'][paus_mask], z_bins,
        weights=np.ones(sum(paus_mask)) * 36**-1 / delta_z,
        histtype='step', lw=2, color='peru')

ax.plot(pd16_g_LF_225['z'], pd16_g_LF_225['counts'] / 0.2, 
        marker='^', ls='', mfc='peru', mec='k', ms=8,
        label='PD16 ($g<22.5$)')
ax.plot(pd16_g_LF_215['z'], pd16_g_LF_215['counts'] / 0.2, 
        marker='^', ls='', mfc='purple', mec='k', ms=8,
        label='PD16 ($g<21.5$)')

ax.set_xlim(2, 4.5)
ax.set_ylim(1e-2, 100)
ax.set_yscale('log')

ax.set_ylabel(r'Number counts [deg$^{-2}\,\Delta z^{-1}$]')
ax.set_xlabel('Redshift')

ax.tick_params(labelsize=17, direction='in', which='both')
ax.yaxis.set_ticks_position('both')
ax.xaxis.set_ticks_position('both')

ax.legend(fontsize=12, framealpha=1, loc='lower left')

fig.savefig('../figures/g_band_LF.pdf', pad_inches=0.1, bbox_inches='tight',
            facecolor='w')
plt.show()